# Calculation of BC sensation level

18-01-2021 GC

In [1]:
from pathlib import Path
import pandas as pd

In [2]:
def read_BC_thresholds():
    # function to read threshold in dB FL
    f_in = '/media/guido/LACIE/Cingle_Guido/Master/Headband/BC_dir_thr_third.xlsx'
    p_in = Path(f_in)   
    df = pd.read_excel(p_in, sheet_name='BC_dir_thr_third', header=0, nrows=85)
    df = df.drop(['Unnamed: 0'], axis=1)
    df = df.fillna(pd.NA)
    return df

In [3]:
def read_HRTF_BAHA():
    # function to read HRTF data for the BC path, collected by Stenfelt
    f_in = '/media/guido/LACIE/Cingle_Guido/Master/Headband/constants.xlsx'
    p_in = Path(f_in)   
    col_to_use = list(range(20))
    df = pd.read_excel(p_in, sheet_name='HRTF_BAHA_Stenfelt', 
                       header=0, nrows=3, usecols=col_to_use)
    df = df.fillna(pd.NA)
    df = df.rename(columns={'Unnamed: 0' : 'Angle'})
    df = df.set_index(['Angle'])
    df = df.round(1)
    s0 = pd.Series(df.iloc[0])
    s90 = pd.Series(df.iloc[1])
    s270 = pd.Series(df.iloc[2])
    return (s0, s90, s270)

In [4]:
def read_BCD_output_65():
    # function to read threshold in dB FL
    f_in = '/media/guido/LACIE/Cingle_Guido/Master/Headband/BCD_band_output.xlsx'
    p_in = Path(f_in)   
    df = pd.read_excel(p_in, sheet_name='BCD_output_65', header=0, nrows=85)
    df = df.drop(['Unnamed: 0'], axis=1)
    df = df.fillna(pd.NA)
    return df

In [5]:
def read_BCD_output_55():
    # function to read threshold in dB FL
    f_in = '/media/guido/LACIE/Cingle_Guido/Master/Headband/BCD_band_output.xlsx'
    p_in = Path(f_in)   
    df = pd.read_excel(p_in, sheet_name='BCD_output_55', header=0, nrows=85)
    df = df.drop(['Unnamed: 0'], axis=1)
    df = df.fillna(pd.NA)
    return df

In [6]:
def sl_65():
    # return list with BC sensation levels for 0, 90 and 270 degrees
    bcd_out = read_BCD_output_65()
    info = bcd_out[['Study_ID', 'Device']]
    bcd_out = bcd_out.drop(columns=['Study_ID', 'Device'])
    
    bc_thr = read_BC_thresholds()
    bc_thr = bc_thr.drop(columns=['Study_ID', 'Device'])
    
    l = list()
    
    for i in range(len(read_HRTF_BAHA())):
        sl = read_HRTF_BAHA()[i] + bcd_out - bc_thr
        idx = (read_HRTF_BAHA()[i]).index
        sl = sl.reindex(columns=idx)
        sl = pd.concat([info, sl], axis=1)
        # set low f SL = SL at 250 Hz
        for f in ['125_Hz', '160_Hz', '200_Hz']: sl[f] = sl['250_Hz']   
        
        l.append(sl)
    return l

In [7]:
def sl_55():
    # return list with BC sensation levels for 0, 90 and 270 degrees
    bcd_out = read_BCD_output_55()
    info = bcd_out[['Study_ID', 'Device']]
    bcd_out = bcd_out.drop(columns=['Study_ID', 'Device'])
    
    bc_thr = read_BC_thresholds()
    bc_thr = bc_thr.drop(columns=['Study_ID', 'Device'])
    
    l = list()
    
    for i in range(len(read_HRTF_BAHA())):
        sl = read_HRTF_BAHA()[i] + bcd_out - bc_thr
        idx = (read_HRTF_BAHA()[i]).index
        sl = sl.reindex(columns=idx)
        sl = pd.concat([info, sl], axis=1)
        for f in ['125_Hz', '160_Hz', '200_Hz']: sl[f] = sl['250_Hz']   
        
        l.append(sl)
    return l

In [8]:
sl65 = sl_65()
sl55 = sl_55()

In [9]:
# write results to xls file in Master directory
fout = '/media/guido/LACIE/Cingle_Guido/Master/Headband/SL_BC.xlsx'
pout = Path(fout)
with pd.ExcelWriter(pout) as writer:
    sl65[0].to_excel(writer, sheet_name='BC_SL_65dB_0deg')
    sl65[1].to_excel(writer, sheet_name='BC_SL_65dB_90deg')
    sl65[2].to_excel(writer, sheet_name='BC_SL_65dB_270deg')
    sl55[0].to_excel(writer, sheet_name='BC_SL_55dB_0deg')
    sl55[1].to_excel(writer, sheet_name='BC_SL_55dB_90deg')
    sl55[2].to_excel(writer, sheet_name='BC_SL_55dB_270deg')